In [1]:
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

from common import *
from ViT import ViT, ViT_Extractor
from GaitData import GaitData
import pandas as pd
from IPython.display import display
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = len(TRAIN_LABEL)
ViT_model = ViT(img_size=64,
            in_chanels=1,
            patch_size=32,
            embed_dim=512,
            num_encoders=6,
            num_heads=8,
            hidden_dim=1024,
            dropout=0.2,
            num_classes=num_classes)

ViT_model.load_state_dict(torch.load('GaitViT_model.pth'))
model = ViT_Extractor(ViT_model).to(device)

In [3]:
def eval(model, gallery_loader, probe_loader, device):
    model.eval()
    
    gallery_features = []
    gallery_labels = []

    with torch.no_grad():
        for inputs, labels in gallery_loader:
            inputs = inputs.to(device)
            labels = labels.to(device) 
            features = model(inputs)
            gallery_features.append(features)
            gallery_labels.append(labels)

        gallery_features = torch.cat(gallery_features, dim=0) 
        gallery_labels = torch.cat(gallery_labels, dim=0)      

        correct = 0
        total = 0

        for inputs, labels in probe_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)  

            features = model(inputs)
            dists = torch.cdist(features, gallery_features)  

            nearest_indices = dists.argmin(dim=1)  
            predicted_labels = gallery_labels[nearest_indices]  

            correct += (predicted_labels == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return accuracy

# GALLERY KHÔNG CHIA GÓC

In [5]:
gallery = GaitData('GEI.pt', TEST_LABEL, TEST_GALLERY, map_label=MAP_TEST_LABEL)
gallery_loader = DataLoader(gallery, batch_size=len(gallery), shuffle=False)

row_accuracies = []
for agl_probe in ANGLES:
    probe = GaitData('GEI.pt', TEST_LABEL, TEST_PROBE, [agl_probe], map_label=MAP_TEST_LABEL)
    probe_loader = DataLoader(probe, batch_size=len(probe), shuffle=False)

    accuracy = eval(model, gallery_loader, probe_loader, device)
    row_accuracies.append(accuracy)

df = pd.DataFrame([row_accuracies], index=["All"], columns=ANGLES)

print("Accuracy matrix (Gallery: All views combined, Probe: each angle):")
df_style = df.style.background_gradient(cmap="Blues").format("{:.4f}")
display(df_style)

Accuracy matrix (Gallery: All views combined, Probe: each angle):


,000,018,036,054,072,090,108,126,144,162,180
All,0.6577,0.6637,0.6607,0.6429,0.6280,0.5744,0.5833,0.6518,0.6637,0.6637,0.6607


# PROBE: NM-05, NM-06

In [4]:
accuracy_matrix = []

for agl_gallery in ANGLES:
    gallery = GaitData('GEI.pt', TEST_LABEL, TEST_GALLERY, [agl_gallery], map_label=MAP_TEST_LABEL)
    gallery_loader = DataLoader(gallery, batch_size=len(gallery), shuffle=False)
    
    row_accuracies = []
    for agl_probe in ANGLES:
        probe = GaitData('GEI.pt', TEST_LABEL, TEST_PROBE_NORMAL, [agl_probe], map_label=MAP_TEST_LABEL)
        probe_loader = DataLoader(probe, batch_size=len(probe), shuffle=False)
        
        accuracy = eval(model, gallery_loader, probe_loader, device)
        row_accuracies.append(accuracy)
    
    accuracy_matrix.append(row_accuracies)

df = pd.DataFrame(accuracy_matrix, index=ANGLES, columns=ANGLES)

print("Accuracy matrix (Gallery: rows, Probe: columns):")

df_style = df.style.background_gradient(cmap="Blues").format("{:.4f}")
display(df_style)
acc_array = df.to_numpy().astype(np.float32)
mask = np.eye(len(ANGLES), dtype=bool)
acc_array[mask] = np.nan
df_no_diag = pd.DataFrame(acc_array, index=ANGLES, columns=ANGLES)
column_means = df_no_diag.mean(skipna=True)
print("Mean accuracy per column (excluding diagonal):")
print(column_means.to_frame(name="Mean Accuracy (no diagonal)").to_string(formatters={"Mean Accuracy (no diagonal)": "{:.4f}".format}))

Accuracy matrix (Gallery: rows, Probe: columns):


,000,018,036,054,072,090,108,126,144,162,180
000,0.9911,0.9107,0.6786,0.4375,0.2857,0.1786,0.2143,0.2321,0.3571,0.5536,0.7232
018,0.9196,1.0000,0.9821,0.7054,0.3750,0.3393,0.3839,0.3929,0.4643,0.5268,0.5536
036,0.7054,0.9375,0.9911,0.9196,0.7321,0.5536,0.5089,0.5893,0.5536,0.5536,0.4911
054,0.3571,0.6339,0.8929,1.0000,0.8929,0.8036,0.7946,0.7054,0.6250,0.3750,0.2321
072,0.1964,0.3393,0.6518,0.8214,0.9643,0.9286,0.8929,0.8125,0.6250,0.3750,0.1518
090,0.1875,0.2946,0.4732,0.7589,0.9643,0.9554,0.9464,0.7946,0.5893,0.3393,0.1518
108,0.1429,0.3125,0.5000,0.7946,0.9286,0.9286,0.9554,0.9464,0.7500,0.4643,0.1875
126,0.2411,0.3839,0.6071,0.6875,0.8482,0.7857,0.9107,0.9821,0.9643,0.6339,0.3214
144,0.3571,0.5179,0.5893,0.6875,0.5804,0.5893,0.7143,0.9196,0.9821,0.8571,0.4821
162,0.5982,0.7321,0.6339,0.4464,0.3304,0.3571,0.3839,0.5357,0.8571,0.9732,0.8750


Mean accuracy per column (excluding diagonal):
    Mean Accuracy (no diagonal)
000                      0.4473
018                      0.5723
036                      0.6500
054                      0.6580
072                      0.6116
090                      0.5652
108                      0.5893
126                      0.6187
144                      0.6214
162                      0.5580
180                      0.4170


# PROBE: BG-01, BG-02

In [5]:
accuracy_matrix = []

for agl_gallery in ANGLES:
    gallery = GaitData('GEI.pt', TEST_LABEL, TEST_GALLERY, [agl_gallery], map_label=MAP_TEST_LABEL)
    gallery_loader = DataLoader(gallery, batch_size=len(gallery), shuffle=False)
    
    row_accuracies = []
    for agl_probe in ANGLES:
        probe = GaitData('GEI.pt', TEST_LABEL, TEST_PROBE_BAG, [agl_probe], map_label=MAP_TEST_LABEL)
        probe_loader = DataLoader(probe, batch_size=len(probe), shuffle=False)
        
        accuracy = eval(model, gallery_loader, probe_loader, device)
        row_accuracies.append(accuracy)
    
    accuracy_matrix.append(row_accuracies)

df = pd.DataFrame(accuracy_matrix, index=ANGLES, columns=ANGLES)

print("Accuracy matrix (Gallery: rows, Probe: columns):")

df_style = df.style.background_gradient(cmap="Blues").format("{:.4f}")
display(df_style)
acc_array = df.to_numpy().astype(np.float32)
mask = np.eye(len(ANGLES), dtype=bool)
acc_array[mask] = np.nan
df_no_diag = pd.DataFrame(acc_array, index=ANGLES, columns=ANGLES)
column_means = df_no_diag.mean(skipna=True)
print("Mean accuracy per column (excluding diagonal):")
print(column_means.to_frame(name="Mean Accuracy (no diagonal)").to_string(formatters={"Mean Accuracy (no diagonal)": "{:.4f}".format}))

Accuracy matrix (Gallery: rows, Probe: columns):


,000,018,036,054,072,090,108,126,144,162,180
000,0.7143,0.5625,0.3214,0.2054,0.1429,0.0893,0.1518,0.1964,0.2411,0.4375,0.5268
018,0.6518,0.6964,0.5179,0.3482,0.1696,0.1607,0.1339,0.3036,0.3036,0.4286,0.4286
036,0.4107,0.6875,0.6964,0.4732,0.3304,0.1786,0.2232,0.3214,0.3482,0.3839,0.3125
054,0.2679,0.3929,0.6607,0.6071,0.5089,0.3750,0.3661,0.4196,0.4375,0.2679,0.1786
072,0.1250,0.2679,0.4286,0.5089,0.6250,0.5089,0.4911,0.5268,0.3482,0.1875,0.1071
090,0.1518,0.2321,0.3571,0.4821,0.5536,0.4821,0.5089,0.4643,0.3304,0.2321,0.1786
108,0.1429,0.2500,0.3661,0.5446,0.5357,0.4911,0.5536,0.6964,0.4643,0.2589,0.2054
126,0.1429,0.2768,0.3839,0.4018,0.3661,0.3393,0.4911,0.7143,0.5357,0.3839,0.2500
144,0.3036,0.4018,0.3929,0.3304,0.2143,0.1607,0.3125,0.6161,0.6518,0.5536,0.3929
162,0.4286,0.5179,0.3393,0.1429,0.1161,0.1250,0.1875,0.2946,0.4464,0.6607,0.5804


Mean accuracy per column (excluding diagonal):
    Mean Accuracy (no diagonal)
000                      0.3196
018                      0.4018
036                      0.4036
054                      0.3589
072                      0.3036
090                      0.2500
108                      0.3000
126                      0.4036
144                      0.3723
162                      0.3643
180                      0.3161


# PROBE: CL-01, CL-02

In [6]:
accuracy_matrix = []

for agl_gallery in ANGLES:
    gallery = GaitData('GEI.pt', TEST_LABEL, TEST_GALLERY, [agl_gallery], map_label=MAP_TEST_LABEL)
    gallery_loader = DataLoader(gallery, batch_size=len(gallery), shuffle=False)
    
    row_accuracies = []
    for agl_probe in ANGLES:
        probe = GaitData('GEI.pt', TEST_LABEL, TEST_PROBE_CLOTHES, [agl_probe], map_label=MAP_TEST_LABEL)
        probe_loader = DataLoader(probe, batch_size=len(probe), shuffle=False)
        
        accuracy = eval(model, gallery_loader, probe_loader, device)
        row_accuracies.append(accuracy)
    
    accuracy_matrix.append(row_accuracies)

df = pd.DataFrame(accuracy_matrix, index=ANGLES, columns=ANGLES)

print("Accuracy matrix (Gallery: rows, Probe: columns):")

df_style = df.style.background_gradient(cmap="Blues").format("{:.4f}")
display(df_style)
acc_array = df.to_numpy().astype(np.float32)
mask = np.eye(len(ANGLES), dtype=bool)
acc_array[mask] = np.nan
df_no_diag = pd.DataFrame(acc_array, index=ANGLES, columns=ANGLES)
column_means = df_no_diag.mean(skipna=True)
print("Mean accuracy per column (excluding diagonal):")
print(column_means.to_frame(name="Mean Accuracy (no diagonal)").to_string(formatters={"Mean Accuracy (no diagonal)": "{:.4f}".format}))

Accuracy matrix (Gallery: rows, Probe: columns):


,000,018,036,054,072,090,108,126,144,162,180
000,0.2589,0.2321,0.2589,0.1518,0.0714,0.0625,0.0804,0.1250,0.2143,0.2411,0.3036
018,0.3125,0.3661,0.3036,0.2679,0.1607,0.1429,0.1696,0.2054,0.2321,0.2411,0.2857
036,0.2054,0.2768,0.4107,0.3661,0.2411,0.1964,0.2500,0.2857,0.2411,0.1964,0.1875
054,0.2143,0.2321,0.3661,0.4196,0.3036,0.2768,0.2589,0.2589,0.2143,0.1429,0.1696
072,0.1071,0.1696,0.2946,0.2679,0.4018,0.3482,0.3482,0.2679,0.2143,0.1339,0.1161
090,0.0893,0.1518,0.2500,0.2768,0.3661,0.3393,0.2946,0.2768,0.2143,0.0804,0.0982
108,0.1071,0.1875,0.2500,0.2589,0.3214,0.3571,0.3750,0.3393,0.2857,0.1429,0.1607
126,0.1071,0.1964,0.2411,0.2500,0.2589,0.3036,0.3661,0.3750,0.2857,0.1875,0.1429
144,0.1339,0.2321,0.2500,0.2321,0.2232,0.2321,0.2857,0.4018,0.3750,0.2500,0.1607
162,0.1161,0.2411,0.2589,0.1964,0.1964,0.1875,0.1429,0.1875,0.3125,0.3929,0.2857


Mean accuracy per column (excluding diagonal):
    Mean Accuracy (no diagonal)
000                      0.1598
018                      0.2143
036                      0.2643
054                      0.2384
072                      0.2223
090                      0.2179
108                      0.2268
126                      0.2446
144                      0.2357
162                      0.1830
180                      0.1911


# PROBE: NM-05, NM-06, BG-01, BG-02, CL-01, CL-02

In [7]:
accuracy_matrix = []

for agl_gallery in ANGLES:
    gallery = GaitData('GEI.pt', TEST_LABEL, TEST_GALLERY, [agl_gallery], map_label=MAP_TEST_LABEL)
    gallery_loader = DataLoader(gallery, batch_size=len(gallery), shuffle=False)
    
    row_accuracies = []
    for agl_probe in ANGLES:
        probe = GaitData('GEI.pt', TEST_LABEL, TEST_PROBE, [agl_probe], map_label=MAP_TEST_LABEL)
        probe_loader = DataLoader(probe, batch_size=len(probe), shuffle=False)
        
        accuracy = eval(model, gallery_loader, probe_loader, device)
        row_accuracies.append(accuracy)
    
    accuracy_matrix.append(row_accuracies)

df = pd.DataFrame(accuracy_matrix, index=ANGLES, columns=ANGLES)

print("Accuracy matrix (Gallery: rows, Probe: columns):")

df_style = df.style.background_gradient(cmap="Blues").format("{:.4f}")
display(df_style)
acc_array = df.to_numpy().astype(np.float32)
mask = np.eye(len(ANGLES), dtype=bool)
acc_array[mask] = np.nan
df_no_diag = pd.DataFrame(acc_array, index=ANGLES, columns=ANGLES)
column_means = df_no_diag.mean(skipna=True)
print("Mean accuracy per column (excluding diagonal):")
print(column_means.to_frame(name="Mean Accuracy (no diagonal)").to_string(formatters={"Mean Accuracy (no diagonal)": "{:.4f}".format}))

Accuracy matrix (Gallery: rows, Probe: columns):


,000,018,036,054,072,090,108,126,144,162,180
000,0.6548,0.5685,0.4196,0.2649,0.1667,0.1101,0.1488,0.1845,0.2708,0.4107,0.5179
018,0.6280,0.6875,0.6012,0.4405,0.2351,0.2143,0.2292,0.3006,0.3333,0.3988,0.4226
036,0.4405,0.6339,0.6994,0.5863,0.4345,0.3095,0.3274,0.3988,0.3810,0.3780,0.3304
054,0.2798,0.4196,0.6399,0.6756,0.5685,0.4851,0.4732,0.4613,0.4256,0.2619,0.1935
072,0.1429,0.2589,0.4583,0.5327,0.6637,0.5952,0.5774,0.5357,0.3958,0.2321,0.1250
090,0.1429,0.2262,0.3601,0.5060,0.6280,0.5923,0.5833,0.5119,0.3780,0.2173,0.1429
108,0.1310,0.2500,0.3720,0.5327,0.5952,0.5923,0.6280,0.6607,0.5000,0.2887,0.1845
126,0.1637,0.2857,0.4107,0.4464,0.4911,0.4762,0.5893,0.6905,0.5952,0.4018,0.2381
144,0.2649,0.3839,0.4107,0.4167,0.3393,0.3274,0.4375,0.6458,0.6696,0.5536,0.3452
162,0.3810,0.4970,0.4107,0.2619,0.2143,0.2232,0.2381,0.3393,0.5387,0.6756,0.5804


Mean accuracy per column (excluding diagonal):
    Mean Accuracy (no diagonal)
000                      0.3089
018                      0.3961
036                      0.4393
054                      0.4185
072                      0.3792
090                      0.3443
108                      0.3720
126                      0.4223
144                      0.4098
162                      0.3685
180                      0.3080
